In [ ]:
The algorithm aims to calculate drivetrain parameters based on efficiency maps while evaluating the associated uncertainty.

In [8]:
from pylab import *
import csv
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.optimize import minimize
import datetime

In [9]:
def load_engine_file(file_name):
    """
    Load engine and drivetrain data from a CSV file and filter out invalid efficiency values.

    Parameters:
    file_name (str): Path to the CSV file.

    Returns:
    tuple: Extracted and filtered drivetrain data.
    """
    # Read the CSV file
    df = pd.read_csv(file_name, sep=';')

    # Extract data from the file
    drivetrain = df.iloc[0, 0]
    gears = df.iloc[:, 1].values
    transmission_ratios = df.iloc[:, 2].values
    rpm_values = df.iloc[:, 3].values
    torque_values = df.iloc[:, 4].values
    efficiency_values = df.iloc[:, 5].values
    transfer_ratio = df.iloc[0, 6]
    max_power = df.iloc[0, 7]

    # Filter out rows with efficiency less than or equal to 10
    valid_indices = np.where(efficiency_values > 10)[0]
    gears = gears[valid_indices]
    transmission_ratios = transmission_ratios[valid_indices]
    rpm_values = rpm_values[valid_indices]
    torque_values = torque_values[valid_indices]
    efficiency_values = efficiency_values[valid_indices]

    return drivetrain, gears, transmission_ratios, rpm_values, torque_values, efficiency_values, transfer_ratio, max_power


In [10]:
# Load engine data
name = 'Toyota.csv'
#name = 'Chrysler.csv'
#name = 'Malibu.csv'
drivetrain, gears, transg, rpm, torque, eff, transf, P_max = load_engine_file(name)

# Calculate transmission RPM and torque
rpm_trans = rpm / (transg * transf)
torque_trans = rpm * torque * eff / (100 * rpm_trans)

# Convert RPM to angular velocity
N = rpm * 2 * np.pi / 60
Nt = rpm_trans * 2 * np.pi / 60
Tt = torque_trans

# Define the optimization objective function
def objective_function(parameters):
    n_tr, a = parameters
    eff_model = Tt * Nt / (Tt * Nt / n_tr + a * N / n_tr) * 100
    error = np.abs(eff - eff_model)
    return np.sum(error)

# Initial parameter values and bounds
initial_parameters = [0.98, 1]
parameter_bounds = [(np.max(eff) / 100, 1), (0, 10)]

# Perform optimization
optimization_result = minimize(objective_function, initial_parameters, method='Powell', bounds=parameter_bounds)

# Extract optimal parameters
optimal_parameters = optimization_result.x
n_tr, a = optimal_parameters

# Calculate the optimized efficiency model
eff_model = Tt * Nt / (Tt * Nt / n_tr + a * N / n_tr) * 100

# Calculate and display the error
error_distribution = (eff - eff_model) / eff * 100
mean_error = np.sum(np.abs(error_distribution)) / len(rpm)

print('Optimization Results:')
print('----------------------')
print(f'Mean Error [%]: {mean_error:.2f}')
print(f'a * P_max [J]: {a:.2f}')
print(f'P_max [W]: {P_max:.0f}')
print(f'a [s] (scaled by P_max): {a / P_max:.1e}')
print(f'n_tr: {n_tr:.4f}')


Optimization Results:
----------------------
Mean Error [%]: 1.83
a * P_max [J]: 4.47
P_max [W]: 151000
a [s] (scaled by P_max): 3.0e-05
n_tr: 0.9721
